In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 


from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

import gensim

from gensim import models, corpora

from sklearn.decomposition import TruncatedSVD

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split



In [4]:
data_train_gen = pd.read_csv("/Users/aleksejfilippov/Desktop/techotrack/quora_2/csvs/train.csv")
data_test = pd.read_csv("/Users/aleksejfilippov/Desktop/techotrack/quora_2/csvs/test.csv")
print("Train datasets shape:", data_train_gen.shape)
print("Test datasets shape:", data_test.shape)

Train datasets shape: (1306122, 3)
Test datasets shape: (56370, 2)


# 1.1 Lemmatization, class ratio, getting rid of stopwords.

In [86]:
num = 653826
bad_df = data_train_gen[data_train_gen.target == 1]
good_df = data_train_gen[data_train_gen.target == 0][:num]
data_train = pd.concat([bad_df, good_df])

In [87]:
from sklearn.utils import shuffle
data_train = shuffle(data_train)
data_train = data_train[:136520]

In [88]:
quiestion_words = ['what','when','why','which','who','how', 'whose', 'whome', 'people', 'i', 
                  'n\'t','\'s','like','get','would','would']
stop_signs = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']
stop_words = stopwords.words('english')
for w in quiestion_words:
    stop_words.append(w)
for w in stop_signs:
    stop_words.append(w)

In [89]:
cleaned_questions_train = []
 
for sentence in data_train['question_text']:
    new_sentence = [wordnet_lemmatizer.lemmatize(w).lower() for w in word_tokenize(sentence)]
    new_sentence = [w for w in new_sentence if w not in stop_words]
    new_sentence = [w for w in new_sentence if len(w)>3]
         
    clean = ' '.join(new_sentence)    
   
    cleaned_questions_train.append(clean)

cleaned_questions_test = []
for sentence in data_test['question_text']:
    new_sentence = [wordnet_lemmatizer.lemmatize(w).lower() for w in word_tokenize(sentence)]
    new_sentence = [w for w in new_sentence if w not in stop_words]
    new_sentence = [w for w in new_sentence if len(w)>3]
         
    clean = ' '.join(new_sentence)    
   
    cleaned_questions_test.append(clean)

In [90]:
data_train.insert(loc=0, column="debugged_questions", value=cleaned_questions_train)
data_test.insert(loc=0, column="debugged_questions", value=cleaned_questions_test)

In [92]:
data_train.fillna('', inplace = True)
data_test.fillna('', inplace = True)

In [146]:
train = data_train.ix[:, (1,0, 3)]
test = data_test.ix[:, (1,0)]

/Users/aleksejfilippov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/Users/aleksejfilippov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


# 1.2  word2vec 

In [96]:
train['tokens_list'] = train.debugged_questions.apply(lambda x: x.split())
test['tokens_list'] = test.debugged_questions.apply(lambda x: x.split())

In [13]:
word2vec_path = "GoogleNews-vectors-negative300.bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [105]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens_list'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return embeddings.tolist()



In [108]:
test_embeddings = get_word2vec_embeddings(word2vec, test)
train_embeddings = get_word2vec_embeddings(word2vec, train)

In [110]:
test_embeddings = pd.DataFrame(test_embeddings)
train_embeddings = pd.DataFrame(train_embeddings)

# 2.1 LDA model

In [118]:
# ну или импортирую заранее обученную
ldamodel = models.ldamodel.LdaModel.load("ldamodel3_lkcd")

In [114]:
gen = pd.concat([train[train.columns[[0,1,3]]], test], axis = 0)
tokens = gen.tokens_list.tolist()
dictionary = corpora.Dictionary(tokens)
corpus = []
for token in tokens: corpus.append(dictionary.doc2bow(token))

In [115]:
tetok = test.tokens_list.tolist()
trtok = train.tokens_list.tolist()
# wtok = weird_tok.tolist()
tecor = []
trcor = []
# wecor = []
for token in tetok: tecor.append(dictionary.doc2bow(token))
for token in trtok: trcor.append(dictionary.doc2bow(token))
# for token in wtok: wecor.append(dictionary2.doc2bow(token))

In [117]:
np.random.seed(76543)
%time ldamodel_orig = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=200, passes=3)

## 3.1 Create matrix Theta describing the destribution of topics above documents.

In [120]:
test_topics = ldamodel.get_document_topics(tecor)
train_topics = ldamodel.get_document_topics(trcor)
# weird_topics = ldamodel.get_document_topics(wecor)

In [121]:
nums1 = []
for list in [ldamodel.get_document_topics(corp) for corp in trcor]:
    num1 = []
    for tup in list:
        num1.append(tup)
    nums1.append(num1)

In [122]:
nums2 = []
nn = []
for list in [ldamodel.get_document_topics(corp) for corp in tecor]:
    num2 = []
    for tup in list:
        num2.append(tup)
    nums2.append(num2)

In [123]:
X_ps_tr = np.ndarray([len(trcor), 200])
X_ps_te = np.ndarray([len(tecor), 200])

In [124]:
for i in range(len(nums1)):
    if i == 136523: pass
    for j in range(len(nums1[i])):
        X_ps_tr[i, nums1[i][j][0]] = nums1[i][j][1]
        
        
for i in range(len(nums2)):
    for j in range(len(nums2[i])):
        X_ps_te[i, nums2[i][j][0]] = nums2[i][j][1]

In [126]:
X_ps_te = pd.DataFrame(X_ps_te)
X_ps_tr = pd.DataFrame(X_ps_tr)

In [151]:
train.index = pd.RangeIndex(0, train.shape[0])

In [165]:
trainig = pd.concat([train.qid, train.target, train_embeddings, X_ps_tr], axis = 1)
testing = pd.concat([test.qid, test_embeddings, X_ps_te], axis = 1)

In [174]:
testing.head()

,qid,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,00014894849d00ba98a9,0.012553,0.043992,-0.030441,0.000315,-0.093994,0.026571,0.015569,-0.117981,0.156372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000156468431f09b3cae,0.011292,0.013550,-0.113220,0.278564,-0.031006,0.019897,0.074738,-0.157593,-0.014236,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000227734433360e1aae,-0.031982,-0.026709,0.134375,0.013586,-0.016016,-0.015918,-0.046609,-0.020842,0.155200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0005e06fbe3045bd2a92,0.006824,0.006291,0.029579,0.098839,-0.109790,0.030280,0.126740,-0.022961,0.186877,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00068a0f7f41f50fc399,-0.140299,0.013590,0.075195,0.100423,0.008138,0.076660,-0.029948,-0.061646,-0.031657,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
lsa = TruncatedSVD(n_components=100)
lsa.fit(trainig[trainig.columns[2:]])
lsa_scores_train = lsa.transform(trainig[trainig.columns[2:]])
lsa_scores_test = lsa.transform(testing[testing.columns[1:]])

In [160]:
lsa_scores_train = pd.DataFrame(lsa_scores_train)
lsa_scores_test = pd.DataFrame(lsa_scores_test)

In [168]:
y = trainig.target

In [172]:
clf = DecisionTreeClassifier(max_depth = 12, min_samples_leaf = 20)
clf.fit(lsa_scores_train,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=12,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [178]:
sub_df = pd.DataFrame({'qid':testing.qid.values})
sub_df['prediction'] = clf.predict(lsa_scores_test)
sub_df.to_csv('submission.csv', index=False)

In [184]:
X_traint, X_testt, y_train, y_test = train_test_split(lsa_scores_train, y, test_size = 0.3, random_state=42)

In [185]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from collections import OrderedDict
from sklearn.metrics import roc_auc_score, log_loss
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [187]:
grid = OrderedDict((
        ('max_depth',np.arange(12,20,1)),
        ('min_samples_leaf', [20])))

result = {'params':[],'roc_auc_train':[],'roc_auc_valid':[], 'f1_train':[], 'f1_valid':[]}

for param_values in itertools.product(*grid.values()):
    param = dict(zip(grid.keys(),param_values))
    clf = DecisionTreeClassifier(**param)

    clf.fit(lsa_scores_train,y)

    train_pred = clf.predict(X_traint)
    valid_pred = clf.predict(X_testt)

    roc_auc_train = roc_auc_score(y_train,train_pred)
    roc_auc_valid = roc_auc_score(y_test,valid_pred)
    f1_train = f1_score(y_train,train_pred)
    f1_test = f1_score(y_test,valid_pred)
    result['params'].append(param)
    result['roc_auc_train'].append(roc_auc_train)
    result['roc_auc_valid'].append(roc_auc_valid)
    result['f1_train'].append(f1_train)
    result['f1_valid'].append(f1_test)

(pd.DataFrame(result)   
   .style
   .background_gradient('Wistia')
)



,params,roc_auc_train,roc_auc_valid,f1_train,f1_valid
0,"{'max_depth': 12, 'min_samples_leaf': 20}",0.745557,0.745601,0.590453,0.589319
1,"{'max_depth': 13, 'min_samples_leaf': 20}",0.758936,0.757009,0.61472,0.610439
2,"{'max_depth': 14, 'min_samples_leaf': 20}",0.768809,0.766169,0.631735,0.628097
3,"{'max_depth': 15, 'min_samples_leaf': 20}",0.768269,0.765813,0.637441,0.63232
4,"{'max_depth': 16, 'min_samples_leaf': 20}",0.768424,0.765687,0.640603,0.636602
5,"{'max_depth': 17, 'min_samples_leaf': 20}",0.767768,0.766342,0.641688,0.638559
6,"{'max_depth': 18, 'min_samples_leaf': 20}",0.766524,0.765586,0.64241,0.640192
7,"{'max_depth': 19, 'min_samples_leaf': 20}",0.765181,0.76397,0.642119,0.640071
